# Обработка данных с датчика

Необходимо автоматически выявлять и заполнять следующие аномалии в данных
- пропуски
- выбросы 

## Первичный анализ данных за год

### Загрузка и преобразование данных

In [7]:
import pandas as pd
import ast
import json

In [3]:
puid_df = pd.read_csv('data/p9445_2024.csv')

In [4]:
puid_df['keys'] = puid_df['keys'].apply(ast.literal_eval)
puid_df['values'] = puid_df['values'].apply(ast.literal_eval)
puid_df['meta'] = puid_df['meta'].apply(ast.literal_eval)

In [5]:
puid_df[:5]

,ts,__insert_ts,keys,values,meta,anomaly
0,2024-05-10 05:20:00,2024-05-13 05:21:06,"[[volume, c1, c2, c3, c4, c5, c6, c7, speed, o...","[[30, 9, 14, 3, 1, 2, 1, null, 78.53125, 3.847...","[{""direction"":1,""lane"":1}, {""direction"":1,""lan...",[]
1,2024-05-10 05:25:00,2024-05-13 05:21:06,"[[volume, c1, c2, c3, c4, c5, c6, c7, speed, o...","[[30, 14, 14, 2, 0, 0, 0, null, 80.55469, 3.10...","[{""direction"":1,""lane"":1}, {""direction"":1,""lan...",[]
2,2024-05-10 05:30:00,2024-05-13 05:21:06,"[[volume, c1, c2, c3, c4, c5, c6, c7, speed, o...","[[44, 28, 15, 0, 0, 1, 0, null, 78.80859, 4.62...","[{""direction"":1,""lane"":1}, {""direction"":1,""lan...",[]
3,2024-05-10 05:35:00,2024-05-13 05:21:06,"[[volume, c1, c2, c3, c4, c5, c6, c7, speed, o...","[[40, 26, 11, 1, 0, 2, 0, null, 75.41797, 4.60...","[{""direction"":1,""lane"":1}, {""direction"":1,""lan...",[]
4,2024-05-10 05:40:00,2024-05-13 05:21:06,"[[volume, c1, c2, c3, c4, c5, c6, c7, speed, o...","[[36, 18, 14, 1, 0, 2, 1, null, 86.62891, 4.08...","[{""direction"":1,""lane"":1}, {""direction"":1,""lan...",[]


### Парсинг столбцов

- meta - список источников данных. От каждого источника поступает набор данных перечисленных в keys 
- keys - наборы ключей для каждого источника. К примеру если meta длины 4 то keys тоже длины 4 и каждый элемент keys - список, содержащий наблюдаемые параметры
- values - значения параметров. Размерность совпадает с keys

Извлечение метаданных. Они константные для датчика

In [14]:
meta = list(map(json.loads, puid_df['meta'][0]))
meta

[{'direction': 1, 'lane': 1},
 {'direction': 1, 'lane': 2},
 {'direction': 0, 'lane': 1},
 {'direction': 0, 'lane': 2}]

In [18]:
titles = list(map(lambda x: f'{"Прямое" if  x["direction"] == 1 else "Обратное"}, полоса {x["lane"]}', meta))
titles

['Прямое, полоса 1',
 'Прямое, полоса 2',
 'Обратное, полоса 1',
 'Обратное, полоса 2']

Маппинг значений по ключам. Для начала будем использовать только основные параметры
- volume
- speed 
- occupancy

In [59]:
keys = puid_df['keys'][0][0]
general_key_indices = [keys.index('volume'), keys.index('speed'), keys.index('occupancy')]
general_key_indices

[0, 8, 9]

In [63]:
def get_processor(source_index):
    def process_keys(row):
        return pd.Series(map(lambda i: pd.to_numeric(row['values'][source_index][i]), general_key_indices), index=map(lambda i: keys[i], general_key_indices))
    
    return process_keys

params_df =  puid_df.apply(get_processor(0), axis=1)
puid_source_0 = pd.concat([puid_df['ts'], params_df], axis=1)

In [ ]:
puid_source_0 = puid_source_0.apply(pd.to_numeric, errors='coerce')

TypeError: no numeric data to plot